In [366]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [367]:
from collections import defaultdict
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes,svm,linear_model
from sklearn.model_selection import  train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [368]:
#import the data
df=pd.read_csv('C:/Users/Ananth kumar/OneDrive/Desktop/movie_metadata.csv')

In [369]:
df.head()

,color,director_name,num_critic_for_reviews,gross,genres,movie_title,cast_total_facebook_likes,num_user_for_reviews,language,country,content_rating,budget,title_year,movie_facebook_likes
0,Color,James Cameron,723.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,Avatar,4834,3054.0,English,USA,PG-13,237000000.0,2009.0,33000
1,Color,Gore Verbinski,302.0,309404152.0,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,48350,1238.0,English,USA,PG-13,300000000.0,2007.0,0
2,Color,Sam Mendes,602.0,200074175.0,Action|Adventure|Thriller,Spectre,11700,994.0,English,UK,PG-13,245000000.0,2015.0,85000
3,Color,Christopher Nolan,813.0,448130642.0,Action|Thriller,The Dark Knight Rises,106759,2701.0,English,USA,PG-13,250000000.0,2012.0,164000
4,NaN,Doug Walker,NaN,NaN,Documentary,Star Wars: Episode VII - The Force Awakens ...,143,NaN,NaN,NaN,NaN,NaN,NaN,0


In [370]:
df.shape

(5043, 14)

In [371]:
 df.isnull().sum()

color                         19
director_name                104
num_critic_for_reviews        50
gross                        884
genres                         0
movie_title                    0
cast_total_facebook_likes      0
num_user_for_reviews          21
language                      14
country                        5
content_rating               303
budget                       492
title_year                   108
movie_facebook_likes           0
dtype: int64

In [372]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   gross                      4159 non-null   float64
 4   genres                     5043 non-null   object 
 5   movie_title                5043 non-null   object 
 6   cast_total_facebook_likes  5043 non-null   int64  
 7   num_user_for_reviews       5022 non-null   float64
 8   language                   5029 non-null   object 
 9   country                    5038 non-null   object 
 10  content_rating             4740 non-null   object 
 11  budget                     4551 non-null   float64
 12  title_year                 4935 non-null   float64
 13  movie_facebook_likes       5043 non-null   int64

In [373]:
df.describe()

,num_critic_for_reviews,gross,cast_total_facebook_likes,num_user_for_reviews,budget,title_year,movie_facebook_likes
count,4993.000000,4.159000e+03,5043.000000,5022.000000,4.551000e+03,4935.000000,5043.000000
mean,140.194272,4.846841e+07,9699.063851,272.770808,3.975262e+07,2002.470517,7525.964505
std,121.601675,6.845299e+07,18163.799124,377.982886,2.061149e+08,12.474599,19320.445110
min,1.000000,1.620000e+02,0.000000,1.000000,2.180000e+02,1916.000000,0.000000
25%,50.000000,5.340988e+06,1411.000000,65.000000,6.000000e+06,1999.000000,0.000000
50%,110.000000,2.551750e+07,3090.000000,156.000000,2.000000e+07,2005.000000,166.000000
75%,195.000000,6.230944e+07,13756.500000,326.000000,4.500000e+07,2011.000000,3000.000000
max,813.000000,7.605058e+08,656730.000000,5060.000000,1.221550e+10,2016.000000,349000.000000


In [374]:
df.language.value_counts()

language
English       4704
French          73
Spanish         40
Hindi           28
Mandarin        26
German          19
Japanese        18
Cantonese       11
Russian         11
Italian         11
Portuguese       8
Korean           8
Danish           5
Arabic           5
Hebrew           5
Swedish          5
Polish           4
Norwegian        4
Persian          4
Dutch            4
Chinese          3
Thai             3
Icelandic        2
Aboriginal       2
Indonesian       2
Zulu             2
Romanian         2
Dari             2
Panjabi          1
Vietnamese       1
Slovenian        1
Greek            1
Dzongkha         1
Tamil            1
Urdu             1
Telugu           1
Kannada          1
Czech            1
Hungarian        1
Bosnian          1
Filipino         1
Mongolian        1
Maya             1
Aramaic          1
Kazakh           1
Swahili          1
Name: count, dtype: int64

In [375]:
len(df[df.duplicated()])

98

In [376]:
df.drop_duplicates(inplace=True)

In [377]:
def lower_word(word):
    return word.lower()
df['genres']=df['genres'].map(lower_word)

In [378]:
import re
def regex_(raw_text):
    find_html=re.compile('<.*?>')
    clean_text=re.sub(find_html,"",raw_text)
    return clean_text
df.genres=df.genres.apply(lambda x:regex_(x))

In [379]:
df.genres

0       action|adventure|fantasy|sci-fi
1              action|adventure|fantasy
2             action|adventure|thriller
3                       action|thriller
4                           documentary
                     ...               
5038                       comedy|drama
5039       crime|drama|mystery|thriller
5040              drama|horror|thriller
5041               comedy|drama|romance
5042                        documentary
Name: genres, Length: 4945, dtype: object

In [380]:
w_token=WordPunctTokenizer()
df['genres_tokenized']=[w_token.tokenize(t) for t in df['genres']]

In [381]:
df.head()

,color,director_name,num_critic_for_reviews,gross,genres,movie_title,cast_total_facebook_likes,num_user_for_reviews,language,country,content_rating,budget,title_year,movie_facebook_likes,genres_tokenized
0,Color,James Cameron,723.0,760505847.0,action|adventure|fantasy|sci-fi,Avatar,4834,3054.0,English,USA,PG-13,237000000.0,2009.0,33000,"[action, |, adventure, |, fantasy, |, sci, -, fi]"
1,Color,Gore Verbinski,302.0,309404152.0,action|adventure|fantasy,Pirates of the Caribbean: At World's End,48350,1238.0,English,USA,PG-13,300000000.0,2007.0,0,"[action, |, adventure, |, fantasy]"
2,Color,Sam Mendes,602.0,200074175.0,action|adventure|thriller,Spectre,11700,994.0,English,UK,PG-13,245000000.0,2015.0,85000,"[action, |, adventure, |, thriller]"
3,Color,Christopher Nolan,813.0,448130642.0,action|thriller,The Dark Knight Rises,106759,2701.0,English,USA,PG-13,250000000.0,2012.0,164000,"[action, |, thriller]"
4,NaN,Doug Walker,NaN,NaN,documentary,Star Wars: Episode VII - The Force Awakens ...,143,NaN,NaN,NaN,NaN,NaN,NaN,0,[documentary]


In [382]:
tag_map=defaultdict(lambda:wordnet.NOUN)
tag_map['J']=wordnet.ADJ
tag_map['V']=wordnet.VERB
tag_map['R']=wordnet.ADV

In [383]:
for idx,t in enumerate(df.genres_tokenized):
 if idx %100==0:
    print(idx)
word_ls=[]
wordnet_lemmatizer=WordNetLemmatizer()
for word,tag in pos_tag(t):
     if word not in stopwords.words("english") and word.isalpha():
         word_p=wordnet_lemmatizer.lemmatize(word,tag_map[tag[0]])
         word_ls.append(word_p)
df.loc         [idx,"genres_tokenized_cleaned"]=str(word_ls)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [384]:
df.head()

,color,director_name,num_critic_for_reviews,gross,genres,movie_title,cast_total_facebook_likes,num_user_for_reviews,language,country,content_rating,budget,title_year,movie_facebook_likes,genres_tokenized,genres_tokenized_cleaned
0,Color,James Cameron,723.0,760505847.0,action|adventure|fantasy|sci-fi,Avatar,4834,3054.0,English,USA,PG-13,237000000.0,2009.0,33000,"[action, |, adventure, |, fantasy, |, sci, -, fi]",NaN
1,Color,Gore Verbinski,302.0,309404152.0,action|adventure|fantasy,Pirates of the Caribbean: At World's End,48350,1238.0,English,USA,PG-13,300000000.0,2007.0,0,"[action, |, adventure, |, fantasy]",NaN
2,Color,Sam Mendes,602.0,200074175.0,action|adventure|thriller,Spectre,11700,994.0,English,UK,PG-13,245000000.0,2015.0,85000,"[action, |, adventure, |, thriller]",NaN
3,Color,Christopher Nolan,813.0,448130642.0,action|thriller,The Dark Knight Rises,106759,2701.0,English,USA,PG-13,250000000.0,2012.0,164000,"[action, |, thriller]",NaN
4,NaN,Doug Walker,NaN,NaN,documentary,Star Wars: Episode VII - The Force Awakens ...,143,NaN,NaN,NaN,NaN,NaN,NaN,0,[documentary],NaN


In [385]:
df['genres_tokenized_cleaned'].isna().sum()

4944

In [386]:
X_train, X_test, y_train, y_test = train_test_split(df['genres_tokenized_cleaned'], df['language'], test_size=0.25,random_state=0)

In [387]:
print("X_train shape:",X_train.shape)
print("X_test.shape:",X_train.shape)

X_train shape: (3708,)
X_test.shape: (3708,)


In [388]:
y_train

4544     English
627      English
628      English
2222     English
2963     English
          ...   
5029    Japanese
3307     English
1668     English
2636     English
2763     English
Name: language, Length: 3708, dtype: object

In [389]:
df.genres.unique()

array(['action|adventure|fantasy|sci-fi', 'action|adventure|fantasy',
       'action|adventure|thriller', 'action|thriller', 'documentary',
       'action|adventure|sci-fi', 'action|adventure|romance',
       'adventure|animation|comedy|family|fantasy|musical|romance',
       'adventure|family|fantasy|mystery', 'action|adventure',
       'action|adventure|western', 'action|adventure|family|fantasy',
       'action|adventure|comedy|family|fantasy|sci-fi',
       'adventure|fantasy', 'action|adventure|drama|history',
       'adventure|family|fantasy', 'action|adventure|drama|romance',
       'drama|romance', 'action|adventure|sci-fi|thriller',
       'action|adventure|fantasy|romance',
       'action|adventure|fantasy|sci-fi|thriller',
       'adventure|animation|comedy|family|fantasy',
       'adventure|animation|comedy|family|sport', 'action|crime|thriller',
       'action|adventure|horror|sci-fi|thriller',
       'adventure|animation|family|sci-fi',
       'action|comedy|crime|thrille

In [390]:
enc=LabelEncoder()
X_train=enc.fit_transform(X_train)
X_test=enc.transform(X_test)

In [391]:
df['genres_tokenized_cleaned'].fillna('', inplace=True)
X_train = X_train.astype(str)

In [392]:
df['genres_tokenized_cleaned'] = df['genres_tokenized_cleaned'].str.replace('|', '').str.replace('-', '')


In [393]:
nan_indices_train = y_train[pd.isna(y_train)].index
print(f"NaN values in y_train: {len(nan_indices_train)}")
nan_indices_test = y_test[pd.isna(y_test)].index
print(f"NaN values in y_test: {len(nan_indices_test)}")


NaN values in y_train: 8
NaN values in y_test: 6


In [394]:
y_train.fillna('Unknown', inplace=True)
y_test.fillna('Unknown', inplace=True)


In [395]:
tfidf = TfidfVectorizer(max_features=5000)
tfidf.fit(df.genres_tokenized_cleaned)
X_train=tfidf.transform(y_train)
X_test=tfidf.transform(y_test)


In [396]:


nb=naive_bayes.GaussianNB()
nb.fit(X_train_array,y_train)


GaussianNB()

In [397]:
pred_nb=nb.predict(X_test_array)
print("Accuracy score:",accuracy_score(y_test,pred_nb))

print("confusion matrix:",confusion_matrix(y_test,pred_nb))


Accuracy score: 0.0
confusion matrix: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   2    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1147    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  20    0    0 